In [89]:
import re
import pandas as pd
from pathlib import Path
from collections import defaultdict
import unicodedata


In [90]:
def normalizar(texto: str) -> str:
    texto = texto.upper()
    texto = unicodedata.normalize("NFD", texto)
    texto = texto.encode("ascii", "ignore").decode("utf-8")
    return texto.strip()


In [91]:
MAPA_ASIGNATURAS = {
    "MATEMATICAS": "MAT",
    "MAT-AV": "MAT",
    "MAT": "MAT",
    "MAT-APL": "MAT",
    "MATES": "MAT",
    "MATEMATICAS APLICADAS": "MAT",
    "MATEMATICAS AVANZADAS": "MAT",
    "LENGUA": "LEN",
    "LENGUA CASTELLANA": "LEN",
    "LENGUA CASTELLANA Y LITERATURA": "LEN",
    "INGLES": "ING",
    "ENGLISH": "ING",
    "LENGUA EXTRANJERA INGLES": "ING",
    # FÍSICA Y QUÍMICA
    "FISICA Y QUIMICA": "FYQ",
    "FISICAYQUIMICA": "FYQ",
    "FYQ": "FYQ",
    "FISICA": "FISICA",
    "FIS": "FISICA",
    "QUIMICA": "QUIMICA",
    "QUI": "QUIMICA",
    "BIOLOGIA": "BIO",
    "BIO Y GEO": "BIO",
    "BYG": "BIO",
    "BIO": "BIO",
    "BIOLOGIA Y GEOLOGIA": "BIO",
    "LATIN": "LAT",
    "HISTORIA": "HIST",
    "HISTORIA DEL ARTE": "HISTART",
    "GEOGRAFIA": "GEO",
    "GEOGRAFIA E HISTORIA":"GEH",
    # EDUCACIÓN FÍSICA
    "EDUCACION FISICA": "EF",
    "ED FISICA": "EF",
    "EF": "EF",
    "EDUCACION PLASTICA": "PLAS",
    "PLASTICA": "PLAS",
    "EDUCACION PLASTICA Y VISUAL": "PLAS",
    "EPV": "PLAS",
    "PLAS": "PLAS",
    "FRANCES": "FR",
    "SEGUNDO IDIOMA EXTRANJERO": "FR",
    "FR": "FR",
    "MUSICA":"MUS",
    "HISTORIA DE LA MUSICA": "MUS",
    "HISTORIA DE LA MUSICA Y LA DANZA": "MUS",
    "HISTORIA Y CULTURA DEL ARTE": "HISTART",
    "HISTORIA DEL MUNDO CONTEMPORANEO": "HIST",
    "HISTORIA DE ESPAÑA": "HIST",
    "LITERATURA UNIVERSAL": "LITUNI",
    "LIT UNI": "LITUNI",
    # ECONOMÍA
    "ECONOMIA": "ECO",
    "ECO": "ECO",
    # TECNOLOGÍA
    "TECNOLOGIA": "TEC",
    "TECNO": "TEC",
    "TEC": "TEC",
    # FILOSOFÍA
    "FILOSOFIA": "FIL",
    "FIL": "FIL",
    "AUXILIAR": "AUX",
    "APOYO": "AUX",
    "AUX": "AUX",
    # RELIGIÓN / ORIENTACIÓN / APOYO
    "RELIGION": "REL",
    "ORIENTACION": "ORIENT",
    "VALORES ETICOS": "ORIENT",
    "REL": "REL",
    "AUDIO VISUALES": "AUD",
    "AUD": "AUD",
    "A-V": "AUD"
}

MAPA_CURSOS_TEXTO = {
    "1 ESO": "1E",
    "1ESO": "1E",
    "2 ESO": "2E",
    "2ESO": "2E",
    "3 ESO": "3E",
    "3ESO": "3E",
    "4 ESO": "4E",
    "4ESO": "4E",
    "1 BACH": "1BACH",
    "1BACHILLERATO": "1BACH",
    "2 BACH": "2BACH",
    "2BACHILLERATO": "2BACH",
}


In [92]:
PATRON_ASIGNACION_ESTRICTO = re.compile(
    r"""
    (?P<prof>[A-Z0-9]+)      # Profesor (MAT1, BIO3, etc)
    \s+
    (?P<curso>1E|2E|3E|4E|1BACH|2BACH)
    \s+
    (?P<letra>[A-Z])
    \s+
    (?P<asig>[A-Z\-]+)
    """,
    re.VERBOSE
)



PATRON_RESTRICCION = re.compile(
    r"""
    ^\s*
    (?P<prof>[A-Z0-9]+)
    \s+
    (?P<dia>[1-5])
    \s+
    (?P<hora>[1-6])
    \s*$
    """,
    re.VERBOSE
)




In [93]:
def normalizar_asignatura(asig_raw: str) -> str:
    asig_raw = normalizar(asig_raw)
    return MAPA_ASIGNATURAS.get(asig_raw, asig_raw)


In [94]:
def normalizar_curso_texto(texto: str) -> str:
    for k, v in MAPA_CURSOS_TEXTO.items():
        texto = texto.replace(k, v)
    return texto


In [95]:
def parse_restricciones_txt(ruta_txt: str) -> dict:
    """
    Devuelve un diccionario:
    { "MAT1": "1_4", "BIO4": "5_5", ... }
    """
    restricciones = {}

    with open(ruta_txt, encoding="utf-8") as f:
        for linea in f:
            linea = normalizar(linea)
            if not linea:
                continue

            m = PATRON_RESTRICCION.match(linea)  # ⬅️ MATCH, no search
            if not m:
                print("NO MATCH:", repr(linea))
                continue

            prof = m.group("prof")
            dia = m.group("dia")
            hora = m.group("hora")

            restricciones[prof] = f"{dia}_{hora}"

    return restricciones


In [96]:
def parse_txt(
    ruta_txt: str,
    ruta_restricciones: str | None = None
) -> pd.DataFrame:

    asignaciones = []

    restricciones = {}
    if ruta_restricciones is not None:
        restricciones = parse_restricciones_txt(ruta_restricciones)

    with open(ruta_txt, encoding="utf-8") as f:
        for linea in f:
            linea = normalizar(linea)
            if not linea:
                continue

            m = PATRON_ASIGNACION_ESTRICTO.search(linea)
            if not m:
                continue

            prof = m.group("prof")
            curso = m.group("curso")
            letra = m.group("letra")
            asig = normalizar_asignatura(m.group("asig"))

            asignaciones.append({
                "profesor": prof,
                "curso": curso,
                "letra": letra,
                "grupo": f"{curso}_{letra}",
                "asignatura": asig,
                "restricciones": restricciones.get(prof, "")
            })

    df = pd.DataFrame(asignaciones)

    # Orden profesional
    return df[[
        "profesor", "curso", "letra", "grupo", "asignatura", "restricciones"
    ]]





In [97]:


def generar_csv_apc(
    ruta_txt: str,
    salida="../data/prueba_profesor_grupo_asignatura.csv",
    ruta_restricciones: str | None = None
):
    """
    Genera el CSV profesor–curso–letra–grupo–asignatura
    SIN incluir restricciones.
    """

    df = parse_txt(
        ruta_txt,
        ruta_restricciones=ruta_restricciones
    )

    # Nos quedamos solo con las columnas deseadas y en orden
    columnas = ["profesor", "curso", "letra", "grupo", "asignatura"]

    df = (
        df[columnas]
        .drop_duplicates()
        .sort_values(columnas)
        .reset_index(drop=True)
    )

    Path(salida).parent.mkdir(exist_ok=True)
    df.to_csv(salida, index=False, encoding="utf-8")

    return df



In [98]:


def generar_df_compatibilidad(
    ruta_txt: str,
    salida="../data/prueba_horario_instituto.csv",
    ruta_restricciones: str | None = None,
) -> pd.DataFrame:
    """
    Genera matriz binaria profesor x grupo
    + columna restricciones
    """
    df_parse = parse_txt(
        ruta_txt,
        ruta_restricciones=ruta_restricciones
    )
    # 1. Tabla binaria
    tabla = pd.crosstab(
        df_parse["profesor"],
        df_parse["grupo"]
    )

    tabla = (tabla > 0).astype(int)

    # 2. Restricciones (una por profesor)
    restricciones = (
        df_parse[["profesor", "restricciones"]]
        .drop_duplicates(subset="profesor")
        .set_index("profesor")["restricciones"]
    )

    # 3. Formato final
    tabla.insert(0, "id_profesor", tabla.index)
    tabla["restricciones"] = tabla.index.map(restricciones).fillna("")

    tabla.reset_index(drop=True, inplace=True)

    # 4. Guardar si se pide
    if salida is not None:
        Path(salida).parent.mkdir(exist_ok=True)
        tabla.to_csv(salida, index=False, encoding="utf-8")

    return tabla



In [99]:
generar_csv_apc("../data/informacion_asignacion_profesorado.txt")
generar_df_compatibilidad("../data/informacion_asignacion_profesorado.txt", "../data/prueba_horario_instituto.csv", "../data/restricciones_profesores.txt")

grupo,id_profesor,1BACH_A,1BACH_B,1BACH_C,1BACH_D,1BACH_E,1BACH_F,1E_A,1E_B,1E_C,...,3E_D,3E_E,3E_F,4E_A,4E_B,4E_C,4E_D,4E_E,4E_F,restricciones
0,BIO1,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,1_2
1,BIO2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,
2,BIO3,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,
3,BIO4,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,5_5
4,BIO5,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,TEC3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,
106,TEC4,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,
107,TEC5,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2_6
108,TEC6,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,
